# Gemini Usage

In [29]:
# !pip install google-generativeai==0.8.3

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [1]:
import google.generativeai as genai
import json
from utils import load_prompts
prompts = load_prompts("prompts.yaml")

In [2]:
print(prompts)

{'system': {'general': 'You are an expert at Natural Language Inference (NLI). Your task is to analyze two pieces of text - a premise and a hypothesis - and classify them into NEUTRAL, ENTAILMENT, or CONTRADICTION.'}, 'user': {'general': 'premise: {PREMISE_HERE}, hypothesis: {HYPOTHESIS_HERE}.', 'json_mode': "Generate the classification result in JSON with the key: 'result': str (NEUTRAL, ENTAILMENT, or CONTRADICTION)", 'few': 'USER: premise: {PREMISE_1}, hypothesis: {HYPOTHESIS_1}.\nMODEL: {RESULT_1}\nUSER: premise: {PREMISE_2}, hypothesis: {HYPOTHESIS_2}.\nMODEL: {RESULT_2}\nUSER: premise: {PREMISE_3}, hypothesis: {HYPOTHESIS_3}.\nMODEL:'}, 'mutual': {'few_hint': 'Please first check the following examples.'}}


In [3]:
MY_GOOGLE_API_KEY = "MY_GOOGLE_API_KEY"
genai.configure(api_key=MY_GOOGLE_API_KEY)

In [4]:
MODEL_NAME = "gemini-1.5-pro"
TEMPERATURE = 0

## Example for Classification
- Dataset: SemEval 2014 Task1-2 (3-class classification)

In [5]:
inputs = {
    "premise": "A group of kids is playing in a yard and an old man is standing in the background",
    "hypothesis": "A group of boys in a yard is playing and a man is standing in the background", 
}

In [6]:
system_prompt = prompts["system"]["general"]
user_prompt = prompts["user"]["general"]

In [7]:
print(system_prompt)
print(user_prompt)

You are an expert at Natural Language Inference (NLI). Your task is to analyze two pieces of text - a premise and a hypothesis - and classify them into NEUTRAL, ENTAILMENT, or CONTRADICTION.
premise: {PREMISE_HERE}, hypothesis: {HYPOTHESIS_HERE}.


In [8]:
model = genai.GenerativeModel(
    MODEL_NAME,
    generation_config={"temperature": TEMPERATURE},
    system_instruction=system_prompt,
)

In [9]:
cur_user_prompt = user_prompt.format(
    PREMISE_HERE=inputs["premise"],
    HYPOTHESIS_HERE=inputs["hypothesis"]
)
print(cur_user_prompt)

premise: A group of kids is playing in a yard and an old man is standing in the background, hypothesis: A group of boys in a yard is playing and a man is standing in the background.


In [10]:
response = model.generate_content(cur_user_prompt)

In [11]:
print(response.text)

NEUTRAL.  The premise states "kids," which could include girls. The hypothesis specifies "boys."  The premise says "old man," while the hypothesis just says "man."  While an old man *is* a man, the hypothesis doesn't require the man to be old.  Therefore, the hypothesis is neither entailed nor contradicted by the premise.



In [12]:
num_sys_tokens = str(model.count_tokens(system_prompt)).split(": ")[1].strip()
num_usr_tokens = str(model.count_tokens(cur_user_prompt)).split(": ")[1].strip()
num_input_tokens = int(num_sys_tokens) + int(num_usr_tokens)
print(f"Num of input tokens: {num_input_tokens}")

Num of input tokens: 173


In [13]:
num_output_tokens = str(model.count_tokens(response.text)).split(": ")[1].strip()
print(f"Num of output tokens {num_output_tokens}")

Num of output tokens 119


### Generate structured output in JSON

In [10]:
user_prompt_json = prompts["user"]["general"] + " " + prompts["user"]["json_mode"]

In [11]:
cur_user_prompt = user_prompt_json.format(
    PREMISE_HERE=inputs["premise"],
    HYPOTHESIS_HERE=inputs["hypothesis"]
)
print(cur_user_prompt)

premise: A group of kids is playing in a yard and an old man is standing in the background, hypothesis: A group of boys in a yard is playing and a man is standing in the background. Generate the classification result in JSON with the key: 'result': str (NEUTRAL, ENTAILMENT, or CONTRADICTION)


In [16]:
response = model.generate_content(
    cur_user_prompt,
    generation_config={
        "response_mime_type": "application/json",
    },
)

In [17]:
type(response.text)

str

In [18]:
result_json = json.loads(response.text)
print(type(result_json))
print(result_json)

<class 'dict'>
{'result': 'NEUTRAL'}


### Few-shot

In [31]:
fs_user_prompt = prompts["user"]["few"]

In [32]:
cur_fs_user_prompt = fs_user_prompt.format(
    PREMISE_1="A group of kids is playing in a yard and an old man is standing in the background",
    HYPOTHESIS_1="A group of boys in a yard is playing and a man is standing in the background",
    RESULT_1="{'result': 'NEUTRAL'}",
    PREMISE_2="A man, a woman and two girls are walking on the beach",
    HYPOTHESIS_2="A group of people is on a beach",
    RESULT_2="{'result': 'ENTAILMENT'}",
    PREMISE_3="Two teams are competing in a football match",
    HYPOTHESIS_3="Two groups of people are playing football",  
)
print(cur_fs_user_prompt)

USER: premise: A group of kids is playing in a yard and an old man is standing in the background, hypothesis: A group of boys in a yard is playing and a man is standing in the background.
MODEL: {'result': 'NEUTRAL'}
USER: premise: A man, a woman and two girls are walking on the beach, hypothesis: A group of people is on a beach.
MODEL: {'result': 'ENTAILMENT'}
USER: premise: Two teams are competing in a football match, hypothesis: Two groups of people are playing football.
MODEL:


In [14]:
fs_system_prompt = prompts["system"]["general"] + " " + prompts["mutual"]["few_hint"]
print(fs_system_prompt)

You are an expert at Natural Language Inference (NLI). Your task is to analyze two pieces of text - a premise and a hypothesis - and classify them into NEUTRAL, ENTAILMENT, or CONTRADICTION. Please first check the following examples.


In [29]:
model = genai.GenerativeModel(
    MODEL_NAME,
    generation_config={"temperature": TEMPERATURE},
    system_instruction=fs_system_prompt,
)

In [33]:
response = model.generate_content(
    cur_fs_user_prompt,
    generation_config={
        "response_mime_type": "application/json",
    },
)

In [34]:
result_json = json.loads(response.text)
print(result_json)

{'result': 'ENTAILMENT'}


## Example for Summarization

In [35]:
system_prompt = """你是個中文文本摘要的專家，現在請你對一篇輸入的文章進行摘要。"""

model = genai.GenerativeModel(
    MODEL_NAME,
    generation_config={"temperature": TEMPERATURE},
    system_instruction=system_prompt,
)

In [64]:
input_source_txt = "新华社受权于18日全文播发修改后的《中华人民共和国立法法》，修改后的立法法分为“总则”“法律”“行政法规”“地方性法规、自治条例和单行条例、规章”“适用与备案审查”“附则”等6章，共计105条。"

In [65]:
response = model.generate_content(input_source_txt)

In [66]:
print(response.text)

'新修订的《中华人民共和国立法法》已正式发布，共六章105条，涵盖总则、法律、行政法规、地方性法规及规章、适用与备案审查以及附则等方面。\n'

# Claude Usage

In [ ]:
# !pip install anthropic==0.39.0

In [1]:
import anthropic
import json
import re
from utils import load_prompts
prompts = load_prompts("prompts.yaml")

In [2]:
MY_anthropic_KEY = "MY_anthropic_KEY"
client = anthropic.Anthropic(api_key=MY_anthropic_KEY)

In [3]:
MODEL_NAME = "claude-3-5-sonnet-20241022"
TEMPERATURE = 0

In [4]:
system_prompt = prompts["system"]["general"]

In [5]:
user_prompt_json = prompts["user"]["general"] + " " + prompts["user"]["json_mode"]

In [6]:
inputs = {
    "premise": "A group of kids is playing in a yard and an old man is standing in the background",
    "hypothesis": "A group of boys in a yard is playing and a man is standing in the background", 
}

In [7]:
cur_user_prompt = user_prompt_json.format(
    PREMISE_HERE=inputs["premise"],
    HYPOTHESIS_HERE=inputs["hypothesis"]
)
print(cur_user_prompt)

premise: A group of kids is playing in a yard and an old man is standing in the background, hypothesis: A group of boys in a yard is playing and a man is standing in the background. Generate the classification result in JSON with the key: 'result': str (NEUTRAL, ENTAILMENT, or CONTRADICTION)


In [8]:
response = client.messages.create(
    model=MODEL_NAME,
    max_tokens=1000,
    temperature=TEMPERATURE,
    system=system_prompt,
    messages=[
        {
            "role": "user",
            "content": [{"type": "text", "text": cur_user_prompt}],
        },
    ],
)
result = response.content[0].text

In [9]:
result

'Let me analyze this carefully:\n\n1. Premise mentions "kids" (gender-neutral) while hypothesis specifies "boys" (male)\n2. Premise mentions "old man" while hypothesis just says "man"\n3. Both texts agree on:\n   - Group playing in a yard\n   - Man standing in background\n\nSince the hypothesis specifies "boys" when the premise only mentions "kids" (which could be boys, girls, or mixed), and drops the "old" qualifier for the man, this is a NEUTRAL case. The hypothesis could be true, but it\'s not necessarily entailed by the premise.\n\n{\n    "result": "NEUTRAL"\n}'

In [11]:
# Extract the JSON-like part of the response using regex to match a JSON object
json_match = re.search(r"\{.*?\}", response.content[0].text, re.DOTALL)

In [12]:
json_str = json_match.group(0)  # Extract the JSON string
# Now parse the extracted JSON
result = json.loads(json_str, strict=False)

In [13]:
result

{'result': 'NEUTRAL'}

In [14]:
print(f"Num of input tokens {response.usage.input_tokens}")
print(f"Num of output tokens {response.usage.output_tokens}")

Num of input tokens 122
Num of output tokens 149


## Few-shot

In [17]:
fs_system_prompt = prompts["system"]["general"] + " " + prompts["mutual"]["few_hint"]

In [15]:
fs_user_prompt = prompts["user"]["few"]

In [16]:
cur_fs_user_prompt = fs_user_prompt.format(
    PREMISE_1="A group of kids is playing in a yard and an old man is standing in the background",
    HYPOTHESIS_1="A group of boys in a yard is playing and a man is standing in the background",
    RESULT_1="{'result': 'NEUTRAL'}",
    PREMISE_2="A man, a woman and two girls are walking on the beach",
    HYPOTHESIS_2="A group of people is on a beach",
    RESULT_2="{'result': 'ENTAILMENT'}",
    PREMISE_3="Two teams are competing in a football match",
    HYPOTHESIS_3="Two groups of people are playing football",  
)
print(cur_fs_user_prompt)

USER: premise: A group of kids is playing in a yard and an old man is standing in the background, hypothesis: A group of boys in a yard is playing and a man is standing in the background.
MODEL: {'result': 'NEUTRAL'}
USER: premise: A man, a woman and two girls are walking on the beach, hypothesis: A group of people is on a beach.
MODEL: {'result': 'ENTAILMENT'}
USER: premise: Two teams are competing in a football match, hypothesis: Two groups of people are playing football.
MODEL:


In [18]:
response = client.messages.create(
    model=MODEL_NAME,
    max_tokens=1000,
    temperature=TEMPERATURE,
    system=fs_system_prompt,
    messages=[
        {
            "role": "user",
            "content": [{"type": "text", "text": cur_user_prompt}],
        },
    ],
)
result = response.content[0].text

In [19]:
# Extract the JSON-like part of the response using regex to match a JSON object
json_match = re.search(r"\{.*?\}", response.content[0].text, re.DOTALL)
json_str = json_match.group(0)  # Extract the JSON string
# Now parse the extracted JSON
result = json.loads(json_str, strict=False)

In [20]:
result

{'result': 'NEUTRAL'}

# OpenAI Usage

In [ ]:
# !pip install openai==1.54.5

In [1]:
from openai import OpenAI
import json
from utils import load_prompts
prompts = load_prompts("prompts.yaml")

In [2]:
MY_openai_KEY = "MY_openai_KEY"
client = OpenAI(api_key=MY_openai_KEY)

In [3]:
MODEL_NAME = "gpt-4o"
TEMPERATURE = 0

In [4]:
system_prompt = prompts["system"]["general"]

In [5]:
user_prompt_json = prompts["user"]["general"] + " " + prompts["user"]["json_mode"]

In [6]:
inputs = {
    "premise": "A group of kids is playing in a yard and an old man is standing in the background",
    "hypothesis": "A group of boys in a yard is playing and a man is standing in the background", 
}

In [7]:
cur_user_prompt = user_prompt_json.format(
    PREMISE_HERE=inputs["premise"],
    HYPOTHESIS_HERE=inputs["hypothesis"]
)
print(cur_user_prompt)

premise: A group of kids is playing in a yard and an old man is standing in the background, hypothesis: A group of boys in a yard is playing and a man is standing in the background. Generate the classification result in JSON with the key: 'result': str (NEUTRAL, ENTAILMENT, or CONTRADICTION)


In [8]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    response_format={"type": "json_object"},
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": cur_user_prompt},
    ],
    temperature=TEMPERATURE,
    # max_tokens=max_tokens,
)
result_json = json.loads(response.choices[0].message.content)

In [9]:
result_json

{'result': 'ENTAILMENT'}

In [ ]:
print(f"Num of input tokens {response.usage.prompt_tokens}")
print(f"Num of output tokens {response.usage.completion_tokens}")

### Few-shot

In [10]:
fs_system_prompt = prompts["system"]["general"] + " " + prompts["mutual"]["few_hint"]

In [17]:
# For OpenAI API, when using response_format={"type": "json_object"},
# we must tell the model to output in JSON in the user prompts.
user_prompt = prompts["user"]["general"] + " " + prompts["user"]["json_mode"]

In [18]:
few_shot_inputs = [
    [
        "A group of kids is playing in a yard and an old man is standing in the background",
        "A group of boys in a yard is playing and a man is standing in the background"
    ],
    [
        "A man, a woman and two girls are walking on the beach",
        "A group of people is on a beach"
    ],
]
few_shot_labels = ["NEUTRAL", "ENTAILMENT"]

# inputs without label
real_inputs = [
        "Two teams are competing in a football match",
        "Two groups of people are playing football"
]

In [19]:
messages = [{"role": "system", "content": fs_system_prompt}]
for sents, label in zip(few_shot_inputs, few_shot_labels):
    cur_user_prompt = user_prompt.format(
        PREMISE_HERE=sents[0],
        HYPOTHESIS_HERE=sents[1]
    )
    messages.append({"role": "user", "content": cur_user_prompt})
    messages.append({"role": "assistant", "content": "{"+"'result': "+f"{label}"+"}"})

In [20]:
cur_user_prompt = user_prompt.format(
    PREMISE_HERE=real_inputs[0],
    HYPOTHESIS_HERE=real_inputs[1]
)
messages.append({"role": "user", "content": cur_user_prompt})

In [21]:
messages

[{'role': 'system',
  'content': 'You are an expert at Natural Language Inference (NLI). Your task is to analyze two pieces of text - a premise and a hypothesis - and classify them into NEUTRAL, ENTAILMENT, or CONTRADICTION. Please first check the following examples.'},
 {'role': 'user',
  'content': "premise: A group of kids is playing in a yard and an old man is standing in the background, hypothesis: A group of boys in a yard is playing and a man is standing in the background. Generate the classification result in JSON with the key: 'result': str (NEUTRAL, ENTAILMENT, or CONTRADICTION)"},
 {'role': 'assistant', 'content': "{'result': NEUTRAL}"},
 {'role': 'user',
  'content': "premise: A man, a woman and two girls are walking on the beach, hypothesis: A group of people is on a beach. Generate the classification result in JSON with the key: 'result': str (NEUTRAL, ENTAILMENT, or CONTRADICTION)"},
 {'role': 'assistant', 'content': "{'result': ENTAILMENT}"},
 {'role': 'user',
  'conten

In [22]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    response_format={"type": "json_object"},
    messages=messages,
    temperature=TEMPERATURE,
    # max_tokens=max_tokens,
)
result_json = json.loads(response.choices[0].message.content)

In [23]:
result_json

{'result': 'ENTAILMENT'}